## Import Libraries

In [1]:
import numpy as np
import imageio as iio
import matplotlib.pyplot as plt

## Guassian Stack

In [2]:
def gaussian_filter(sigma, img):
    filt = np.zeros((img.shape[0], img.shape[1], img.shape[2]))
    for i in range(img.shape[2]):
        for j in range(img.shape[1]):
            for k in range(img.shape[0]):
                filt[k,j,i] = np.exp(-((k-img.shape[0]//2)**2+(j-img.shape[1]//2)**2)/sigma**2)
        filt[:,:,i] = filt[:,:,i]/np.max(filt[:,:,i])
    return filt
                

def gaussian_stack(img, n, a):
    '''
    img is the input image,
    n is the number of outputs,
    a can change the speed of bluring
    '''
    G = [] # for save images
    
    r = img[:,:,0]
    g = img[:,:,1]
    b = img[:,:,2]
    
    fr = np.fft.fftshift(np.fft.fft2(r)) # fourier transform of red
    fg = np.fft.fftshift(np.fft.fft2(g)) # fourier transform of green
    fb = np.fft.fftshift(np.fft.fft2(b)) # fourier transform of blue
    
    f = np.dstack((fr,fg,fb))
    
    for i in range(n):
        if i == n-1:
            G.append(img.copy())
        else:
            sigma = (i+1) * a
            filt = gaussian_filter(sigma, f)
            new_fimg = f * filt

            newfr = new_fimg[:,:,0]
            newfg = new_fimg[:,:,1]
            newfb = new_fimg[:,:,2]

            newr = np.fft.ifft2(np.fft.ifftshift(newfr))
            newg = np.fft.ifft2(np.fft.ifftshift(newfg))
            newb = np.fft.ifft2(np.fft.ifftshift(newfb))

            newimg = np.abs(np.dstack((newr, newg, newb)))

            G.append(newimg)
        
    return G

## Laplacian Stack

In [3]:
def laplacian_stack(img, n, a):
    '''
    inputs are same as inputs of gaussian stack function
    '''
    G = gaussian_stack(img, n, a)
    L = []
    L.append(G[0])
    for i in range(n-1):
        L.append(G[i+1]-G[i])
        
    return L

## Blending

In [15]:
def blend(img1, img2, mask, n, a):
    G_mask = gaussian_stack(mask, n, a)
    L1 = laplacian_stack(img1, n, a)
    L2 = laplacian_stack(img2, n, a)
    out = np.zeros(img1.shape)
    for i in range(n):
        out = out+L1[i]*G_mask[i]/np.max(G_mask)+L2[i]*(1-G_mask[i]/np.max(G_mask[i]))
        
    return np.array(255*out/np.max(out), dtype='uint8')


img1 = iio.imread('fig1.jpg')
img2 = iio.imread('fig2.jpg')
mask = iio.imread('fig3.jpg')


img = blend(img1, img2, mask, 5, 10)
iio.imwrite('OstadAsadi_Ronaldo.jpg', img)

img = blend(img2, img1, mask, 5, 10)
iio.imwrite('Ronaldo_OstadAsadi.jpg', img)
